# 0 Setting

In [ ]:
# Parameter Setting
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
folder_name = 'Research-(D5) Synthesized input model'

config = {
    'learning_rate': 1e-4,
    'batch_size': 32,
    'seq_length': 5,

    'shuffle': False,
    'criterion': torch.nn.CrossEntropyLoss(),
    'seed': 42,
    'valid_ratio': 0.2,
    'test_ratio': 0.2,
    'max_length': 512,
    'n_epochs': 3000,
    'early_stop': 100,
}

feature = [
    # X_1
    # 'input_ids',
    # 'attention_mask',
    # 'section_dummy',
    # 'publication_dummy',

    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'today_return_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    'sentiment',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',

    # y
    # '1_day_return',
    # '2_day_return',
    # '3_day_return',
    # '4_day_return',
    # '5_day_return',
    # '1_day_return_cate',
    # '2_day_return_cate',
    # '3_day_return_cate',
    # '4_day_return_cate',
    # '5_day_return_cate',
    # '^DJI', '^DJI_1_day_return', '^GSPC', '^GSPC_1_day_return',
    #    '^NDX', '^NDX_1_day_return', '^IXIC', '^IXIC_1_day_return', '^SOX',
    #    '^SOX_1_day_return',
    # 'excess_return_^DJI',
    # 'excess_return_^DJI_cate',
    # 'excess_return_^GSPC',
    'excess_return_^GSPC_cate',
    # 'excess_return_^NDX',
    # 'excess_return_^NDX_cate',
    # 'excess_return_^IXIC',
    # 'excess_return_^IXIC_cate',
    # 'excess_return_^SOX',
    # 'excess_return_^SOX_cate',


    # Do not mark the datetime, it's for operation
    'datetime',
    ]

# All the news dataset
# time_start = '2016-01-01T00:00:00'
# time_end = '2020-04-02T00:00:00'

time_start = '2016-01-01T00:00:00'
time_end = '2019-12-31T00:00:00'

print(len(feature)-2)

company_list = [
    "Information Technology",
    "Health Care",
    "Financials",
    "Industrials",
    "Consumer Discretionary",
    "Energy",
    "Materials",
    "Communication Services",
    "Utilities",
    "Real Estate",
    "Consumer Staples"
]

process_id = 0  #26

company_name = company_list[process_id]
config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model.csv',
            'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model.ckpt',
            }

7


## (1) Import

In [ ]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install transformers

# Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# PyTorch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import XLNetModel, XLNetTokenizer, BertTokenizer, BertModel

# others
from datetime import datetime, timedelta
from tqdm import tqdm
from torchsummary import summary
import ast

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00


In [ ]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


In [ ]:
df = pd.read_csv(config_2['input_path'])
df = df.sort_values(by='datetime', ascending=True)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,today_return,today_return_cate,...,excess_return_^IXIC_cate,excess_return_^SOX,excess_return_^SOX_cate,excess_return_^NYA,excess_return_^NYA_cate,input_ids,attention_mask,section_dummy,publication_dummy,sentiment
0,38.157607,38.357192,37.749358,38.348122,21638600,0.0,0.0,0.0,0.004993,1,...,0,0.006342,1,-0.007764,0,"[[101, 6522, 1005, 1055, 7069, 8654, 1042, 294...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, ...",0.490476
1,38.466075,38.556795,38.075970,38.248341,16067200,0.0,0.0,0.0,-0.005660,0,...,0,0.013500,1,0.006355,1,"[[101, 13938, 1005, 1055, 8292, 2015, 2355, 25...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, ...",0.476854
2,37.685857,38.030597,37.549772,37.776577,13858400,0.0,0.0,0.0,0.002407,1,...,1,0.001351,1,0.001034,1,"[[101, 2013, 2745, 4027, 2000, 2887, 1043, 102...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, ...",0.480657
3,37.005450,37.486274,36.642561,36.660706,16840700,0.0,0.0,0.0,-0.009316,0,...,1,0.003363,1,0.000485,1,"[[101, 2129, 2146, 2097, 1005, 27830, 3201, 10...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, ...",0.481615
4,36.978219,37.105232,36.325024,36.370384,19233100,0.0,0.0,0.0,-0.016438,0,...,1,0.003958,1,0.004467,1,"[[101, 2131, 5736, 5222, 7485, 1005, 1055, 358...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, ...",0.483336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,77.318843,78.772933,75.855056,76.184647,29242700,0.0,0.0,0.0,-0.014669,0,...,1,0.002589,1,0.009174,1,"[[101, 1996, 5597, 1997, 29458, 1996, 2607, 19...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, ...",0.488077
1066,77.202517,79.577528,76.960168,79.393349,23740100,0.0,0.0,0.0,0.028378,1,...,0,0.004554,1,-0.008134,0,"[[101, 2054, 2017, 2342, 2000, 2113, 2055, 218...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, ...",0.484345
1067,79.209148,80.634157,77.503013,77.910164,19994400,0.0,0.0,0.0,-0.016399,0,...,0,0.005237,1,-0.006226,0,"[[101, 2952, 1997, 1057, 1012, 1055, 1012, 105...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, ...",0.571922
1068,75.215247,76.824443,73.644834,74.197388,27492400,0.0,0.0,0.0,-0.013533,0,...,1,-0.013970,0,0.001834,1,"[[101, 21887, 23350, 2444, 14409, 1024, 1057, ...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, ...",0.354164


In [ ]:
df.publication_dummy[0]

'[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]'

In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Capital Gains', 'today_return', 'today_return_cate', '1_day_return',
       '2_day_return', '3_day_return', '4_day_return', '5_day_return',
       '1_day_return_cate', '2_day_return_cate', '3_day_return_cate',
       '4_day_return_cate', '5_day_return_cate', 'Sma', 'Rsi', 'Kd', 'Ema_12',
       'Ema_26', 'Macd', 'datetime', '^DJI', '^DJI_1_day_return', '^GSPC',
       '^GSPC_1_day_return', '^NDX', '^NDX_1_day_return', '^IXIC',
       '^IXIC_1_day_return', '^SOX', '^SOX_1_day_return', '^NYA',
       '^NYA_1_day_return', 'excess_return_^DJI', 'excess_return_^DJI_cate',
       'excess_return_^GSPC', 'excess_return_^GSPC_cate', 'excess_return_^NDX',
       'excess_return_^NDX_cate', 'excess_return_^IXIC',
       'excess_return_^IXIC_cate', 'excess_return_^SOX',
       'excess_return_^SOX_cate', 'excess_return_^NYA',
       'excess_return_^NYA_cate', 'input_ids', 'attention_mask',
       'section_dummy', '

In [ ]:
# Only contain selected features
df = df[feature]
df.columns

Index(['today_return', 'sentiment', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX',
       'excess_return_^GSPC_cate', 'datetime'],
      dtype='object')

## check data

In [ ]:
df.isnull().sum()

today_return                0
sentiment                   0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

In [ ]:

df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

today_return                0
sentiment                   0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
excess_return_^GSPC_cate    0
datetime                    0
dtype: int64

## (2) Time Period Selection

In [ ]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

(1006, 8)

## (3) Transform str back to list

In [ ]:
df.isnull().sum()

today_return                0
sentiment                   0
^DJI                        0
^GSPC                       0
^NDX                        0
^IXIC                       0
^SOX                        0
excess_return_^GSPC_cate    0
dtype: int64

In [ ]:
# # 将字符串转换回列表的函数
# def string_to_list(s):
#     return ast.literal_eval(s)

# # 将列中的字符串转换回列表
# df['input_ids'] = df['input_ids'].apply(string_to_list)
# df['attention_mask'] = df['attention_mask'].apply(string_to_list)
# df['section_dummy'] = df['section_dummy'].apply(string_to_list)
# df['publication_dummy'] = df['publication_dummy'].apply(string_to_list)

## (4) Train_test_split

In [ ]:
# 1. Set up X, y
to_remove_list = ['datetime', 'excess_return_^GSPC_cate']

# Filter out values in to_remove_list
filtered_list = [x for x in feature if x not in to_remove_list]

X = df[filtered_list]
y = df['excess_return_^GSPC_cate']



# # 1. Set up X, y
# to_remove_list = ['datetime', '1_day_return_cate']

# # Filter out values in to_remove_list
# filtered_list = [x for x in feature if x not in to_remove_list]

# X = df[filtered_list]
# y = df['1_day_return_cate']

In [ ]:
# print(X['section_dummy'])
# print(type(X['section_dummy'][0]))
# temp_array = np.array(X['section_dummy'][0])
# print(temp_array.dtype)

# def int_to_float(int_list):
#   float_list = np.array(int_list, dtype=np.float32)
#   return float_list
# X['section_dummy'] = X['section_dummy'].apply(int_to_float)
# X['publication_dummy'] = X['publication_dummy'].apply(int_to_float)\
# temp_array = np.array(X['section_dummy'][0])
# print(temp_array.dtype)


In [ ]:
# def int_to_float(int_list):
#   float_list = np.array(int_list, dtype=np.float32)
#   return float_list
# X['section_dummy'] = X['section_dummy'].apply(int_to_float)
# X['publication_dummy'] = X['publication_dummy'].apply(int_to_float)

In [ ]:
# Check X, y shape
print('X:', X.shape)
print('y:', y.shape)

X: (1006, 7)
y: (1006,)


In [ ]:
# 2. train_test_split
# val dataset for final examination
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=config['seed'], shuffle=config['shuffle'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=config['seed'], shuffle=config['shuffle'])
X_train


,today_return,sentiment,^DJI,^GSPC,^NDX,^IXIC,^SOX
0,0.004993,0.490476,-0.014739,-0.012531,0.002854,0.001111,0.008979
1,-0.005660,0.476854,0.000651,0.001455,-0.006175,-0.005370,-0.012003
2,0.002407,0.480657,-0.014475,-0.010663,0.007758,0.004570,-0.011092
3,-0.009316,0.481615,-0.022161,-0.021271,-0.009984,-0.009917,-0.010667
4,-0.016438,0.483336,-0.010456,-0.012302,-0.015069,-0.016601,-0.019801
...,...,...,...,...,...,...,...
598,-0.000870,0.487822,0.000298,-0.001612,-0.001512,-0.001358,-0.003253
599,-0.000432,0.513144,0.005234,-0.000870,-0.002137,-0.001661,-0.004453
600,-0.006303,0.502194,-0.008509,-0.005076,-0.005924,-0.005712,-0.001986
601,0.015698,0.497839,0.005215,0.007115,0.015533,0.012397,0.015664


## (5) Scaler

In [ ]:
scale_feature = [
    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'Today_trend_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',
    'sentiment',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',
    # 'datetime'
    ]

def CustomScaler(X_train, X_val, X_test):
  scaler = MinMaxScaler()
  for i in scale_feature:

    # 對特定欄位進行標準化
    X_train_scaled = scaler.fit_transform(X_train[[i]])
    X_val_scaled = scaler.transform(X_val[[i]])
    X_test_scaled = scaler.transform(X_test[[i]])

    # 將標準化後的值重新賦值給 DataFrame
    X_train[i] = X_train_scaled
    X_val[i] = X_val_scaled
    X_test[i] = X_test_scaled

  return X_train, X_val, X_test

X_train, X_val, X_test = CustomScaler(X_train, X_val, X_test)

X_train

,today_return,sentiment,^DJI,^GSPC,^NDX,^IXIC,^SOX
0,0.564830,0.492848,0.355624,0.377236,0.525722,0.515721,0.591670
1,0.442231,0.399203,0.557596,0.578553,0.424253,0.436452,0.374338
2,0.535075,0.425343,0.359087,0.404126,0.580837,0.558035,0.383766
3,0.400162,0.431928,0.258224,0.251419,0.381444,0.380842,0.388167
4,0.318206,0.443761,0.411837,0.380522,0.324291,0.299087,0.293563
...,...,...,...,...,...,...,...
598,0.497355,0.474602,0.552968,0.534407,0.476651,0.485527,0.464971
599,0.502404,0.648677,0.617738,0.545086,0.469628,0.481824,0.452541
600,0.434839,0.573400,0.437380,0.484541,0.427071,0.432268,0.478096
601,0.688021,0.543463,0.617488,0.660028,0.668223,0.653762,0.660923


## (6) Check number

In [ ]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


   Train Count  Train Ratio  Validation Count  Validation Ratio  Test Count  \
1          322     0.533997               113          0.562189         114   
0          281     0.466003                88          0.437811          88   

   Test Ratio  
1    0.564356  
0    0.435644  


In [ ]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test =', X_train.shape[0]/X.shape[0], X_test.shape[0]/X.shape[0], X_val.shape[0]/X.shape[0],
      X_train.shape[0], X_test.shape[0], X_val.shape[0])

Time Period
From: 2016-01-01T00:00:00
To: 2019-12-31T00:00:00 

Sample size: 1006
Feature: Index(['today_return', 'sentiment', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX'], dtype='object') 

Target: excess_return_^GSPC_cate 

Train: Val: Test = 0.5994035785288271 0.20079522862823063 0.19980119284294234 603 202 201


## Benchmark: Mean of excess return

In [3]:
!pip install yfinance
import yfinance as yf
import pandas as pd

In [4]:
def datetime_func(df):
    # Convert the index to a datetime column
    df['datetime'] = df.index

    # Remove the time zone information
    df['datetime'] = pd.to_datetime(df['datetime']).dt.tz_localize(None).dt.date

    return df

# (1) Today Trend
def day_return(df, n, id):
  # Calculate Numerical
  df['today_return'] = (df['Close'] - df['Open']) / df['Open']

  # Calculate categorical
  df['today_return_cate'] = (df['Close'] - df['Open']).apply(lambda x: 1 if x >= 0 else 0)

  # Convert numerical trend to categorical
  df['today_return_cate'] = df['today_return_cate'].astype('category')

  for days in range(1, n+1):
    df[f'{id}_{days}_day_return'] = (df['Close'].shift(-days) / df['Open'].shift(-1) - 1)
  # for days in range(1, n+1):
  #   df[f'{id}_{days}_day_return_cate'] = df[f'{days}_day_return'].apply(lambda x: 1 if x >= 0 else 0)

  return df


In [5]:
ids = ["^DJI", "^GSPC", "^NDX", "^IXIC", "^SOX", '^NYA', 'XLK']  # DJSI

df_list = []

for id in ids:
  dji = yf.Ticker(id)
  df = dji.history(period='Max')
  df = datetime_func(df)
  df = day_return(df, n=1, id=id)
  df[(str(id))] = (df['Close']-df['Open']) / df['Open']
  df = df[[(str(id)), f'{id}_1_day_return', 'datetime']]
  df_list.append(df)

# Merge dataframes
df_index = df_list[0]
for df in df_list[1:]:
  df_index = pd.merge(df_index, df, on='datetime', how='inner')

df_index

,^DJI,^DJI_1_day_return,datetime,^GSPC,^GSPC_1_day_return,^NDX,^NDX_1_day_return,^IXIC,^IXIC_1_day_return,^SOX,^SOX_1_day_return,^NYA,^NYA_1_day_return,XLK,XLK_1_day_return
0,0.006157,0.017393,1998-12-22,0.000607,0.020747,-0.010899,0.028928,-0.010838,0.010822,-0.028475,0.015192,0.000000,0.000000,-0.011090,0.001908
1,0.017393,0.001706,1998-12-23,0.020747,-0.001848,0.028928,-0.007103,0.010822,-0.003804,0.015192,-0.000486,0.000000,0.000000,0.001908,-0.003810
2,0.001706,0.000670,1998-12-24,-0.001848,-0.000636,-0.007103,0.009939,-0.003804,0.000946,-0.000486,-0.013712,0.000000,0.000000,-0.003810,-0.012241
3,0.000670,0.010831,1998-12-28,-0.000636,0.013317,0.009939,-0.000373,0.000946,-0.001218,-0.013712,-0.014890,0.000000,0.000000,-0.012241,-0.001898
4,0.010831,-0.004972,1998-12-29,0.013317,-0.007956,-0.000373,-0.006768,-0.001218,-0.006597,-0.014890,0.019446,0.000000,0.000000,-0.001898,0.001912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6243,0.004496,0.003774,2023-10-16,0.007199,0.006437,0.007855,0.007512,0.008486,0.008486,0.011698,0.013097,0.010494,0.003673,0.007933,0.008030
6244,0.003774,-0.008692,2023-10-17,0.006437,-0.009811,0.007512,-0.007524,0.008486,-0.009329,0.013097,-0.000404,0.003673,-0.013925,0.008030,-0.003269
6245,-0.008692,-0.007584,2023-10-18,-0.009811,-0.010034,-0.007524,-0.013217,-0.009329,-0.012626,-0.000404,-0.022327,-0.013925,-0.008391,-0.003269,-0.009680
6246,-0.007584,-0.007133,2023-10-19,-0.010034,-0.011627,-0.013217,-0.012995,-0.012626,-0.013221,-0.022327,-0.014321,-0.008391,-0.010730,-0.009680,-0.016439


In [6]:
for id in ids:
  df_index[f'excess_return_{id}'] = df_index['XLK_1_day_return'] - df_index[f'{id}_1_day_return']
  df_index[f'excess_return_{id}_cate'] = df_index[f'excess_return_{id}'].apply(lambda x: 1 if x >= 0 else 0)
df_index

,^DJI,^DJI_1_day_return,datetime,^GSPC,^GSPC_1_day_return,^NDX,^NDX_1_day_return,^IXIC,^IXIC_1_day_return,^SOX,...,excess_return_^NDX,excess_return_^NDX_cate,excess_return_^IXIC,excess_return_^IXIC_cate,excess_return_^SOX,excess_return_^SOX_cate,excess_return_^NYA,excess_return_^NYA_cate,excess_return_XLK,excess_return_XLK_cate
0,0.006157,0.017393,1998-12-22,0.000607,0.020747,-0.010899,0.028928,-0.010838,0.010822,-0.028475,...,-0.027020,0,-0.008914,0,-0.013284,0,0.001908,1,0.0,1
1,0.017393,0.001706,1998-12-23,0.020747,-0.001848,0.028928,-0.007103,0.010822,-0.003804,0.015192,...,0.003293,1,-0.000005,0,-0.003323,0,-0.003810,0,0.0,1
2,0.001706,0.000670,1998-12-24,-0.001848,-0.000636,-0.007103,0.009939,-0.003804,0.000946,-0.000486,...,-0.022180,0,-0.013187,0,0.001471,1,-0.012241,0,0.0,1
3,0.000670,0.010831,1998-12-28,-0.000636,0.013317,0.009939,-0.000373,0.000946,-0.001218,-0.013712,...,-0.001525,0,-0.000680,0,0.012992,1,-0.001898,0,0.0,1
4,0.010831,-0.004972,1998-12-29,0.013317,-0.007956,-0.000373,-0.006768,-0.001218,-0.006597,-0.014890,...,0.008680,1,0.008509,1,-0.017534,0,0.001912,1,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6243,0.004496,0.003774,2023-10-16,0.007199,0.006437,0.007855,0.007512,0.008486,0.008486,0.011698,...,0.000518,1,-0.000455,0,-0.005067,0,0.004357,1,0.0,1
6244,0.003774,-0.008692,2023-10-17,0.006437,-0.009811,0.007512,-0.007524,0.008486,-0.009329,0.013097,...,0.004255,1,0.006061,1,-0.002864,0,0.010657,1,0.0,1
6245,-0.008692,-0.007584,2023-10-18,-0.009811,-0.010034,-0.007524,-0.013217,-0.009329,-0.012626,-0.000404,...,0.003537,1,0.002946,1,0.012647,1,-0.001289,0,0.0,1
6246,-0.007584,-0.007133,2023-10-19,-0.010034,-0.011627,-0.013217,-0.012995,-0.012626,-0.013221,-0.022327,...,-0.003444,0,-0.003218,0,-0.002118,0,-0.005708,0,0.0,1
